https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

https://github.com/keras-team/keras/issues/1538

In [1]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.3

from keras.backend.tensorflow_backend import set_session
set_session(tf.Session(config=config))

d:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [4]:
gen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        rescale=1./255)

https://keras.io/preprocessing/image/

https://keras.io/applications/#mobilenet

In [5]:
batch_size=16

In [6]:
train_generator =  gen.flow_from_directory(
    directory = 'train/train',
    target_size=(224,224),
    batch_size=batch_size,
    class_mode='binary',
    #save_to_dir='preview'
)

Found 20000 images belonging to 2 classes.


In [7]:
validation_generator = gen.flow_from_directory(
        directory = 'train/validation',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 2500 images belonging to 2 classes.


In [8]:
test_generator = gen.flow_from_directory(
        directory = 'train/test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 2500 images belonging to 2 classes.


### simple model

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224,224,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

In [10]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 222, 222, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 109, 109, 32)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 64)        18496     
__________

In [12]:
# model.fit_generator?

In [13]:
model.fit_generator(
        train_generator,
        steps_per_epoch=20000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=2500 // batch_size)
#model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/10
1250/1250 [==============================] - 235s 188ms/step - loss: 0.6743 - acc: 0.6077 - val_loss: 0.6296 - val_acc: 0.6242
Epoch 2/10
1250/1250 [==============================] - 232s 186ms/step - loss: 0.6206 - acc: 0.6751 - val_loss: 0.5705 - val_acc: 0.7118
Epoch 3/10
1250/1250 [==============================] - 233s 187ms/step - loss: 0.6113 - acc: 0.6877 - val_loss: 0.5963 - val_acc: 0.6683
Epoch 4/10
1250/1250 [==============================] - 234s 187ms/step - loss: 0.6004 - acc: 0.6965 - val_loss: 0.5860 - val_acc: 0.6940
Epoch 5/10
1250/1250 [==============================] - 236s 189ms/step - loss: 0.5929 - acc: 0.7018 - val_loss: 0.5702 - val_acc: 0.7138
Epoch 6/10
1250/1250 [==============================] - 251s 201ms/step - loss: 0.5843 - acc: 0.7050 - val_loss: 0.5724 - val_acc: 0.7126
Epoch 7/10
1250/1250 [==============================] - 248s 199ms/step - loss: 0.5768 - acc: 0.7173 - val_loss: 0.5490 - val_acc: 0.7238
Epoch 8/10
1250/1250 [============